# business_rules

##### Given a dictionary or Dataframe, produce an output given a set of business rules. 



#### Resources
+ [Google BigQuery](https://console.cloud.google.com/bigquery?project=algomosaic-nyc&p=algomosaic-nyc&page=project)
+ [Google Storage](https://console.cloud.google.com/storage/browser/algom-trading?project=algomosaic-nyc)
+ [Google Drive](https://drive.google.com/drive/folders/1ou6vEWHHeVI0IIjdvj-zGYrCQ7KHRYRm)
+ [Trade Journal: Watchlist](https://docs.google.com/spreadsheets/d/1KXl8GxHlveYT5d3UifVwz-5-oxMyikgSfoZ1evIobmc/edit#gid=1679258225)

<br>



In [1]:
import sys
sys.path.append('/home/jovyan/algom-trading/src')

In [8]:
# helper functions
def get_ror(df, interval_list=[1,2,3,4,5,10,15,20,25,30]):
    df = df.sort_values(by='Date')
    for i in interval_list:
        var = 'ROR_n{}'.format(i)
        df[var] = np.log(df['Close'].shift(i) / df['Close'])
    return df


def trim_backtest_results(df):
    # view a simplified version of the backtest output
    base_list = [
        'backtest_execution_id', 'backtest_executed', 'model_execution_id',
        'As_of_Date', 'Date', 'Ticker', 'Week', 'Month', 'Quarter', 'Year', 'Day_of_Week', 'Day_of_Year',
        'High', 'Low', 'Open', 'Close', 'Volume',
        'trade_response', 'commission', 'amount', 'trade', 'account', 'quantity',
        'buy_trades', 'sell_trades', 'buy_volume', 'sell_volume', 'account_change'
    ]
    df = backtest.df
    new_list = [c for c in list(df) if (c in base_list or 'ROR_n' in c)]
    df_trimmed = df[new_list]
    return df_trimmed

In [2]:
from data_mgmt import bigquery_data as bqd
df = bqd.query_to_df(
    query="""
    select
    partition_date,
    ticker_datetime,
    ticker,
    exchange,
    `interval`,
    close,
    high,
    low,
    open,
    volume_base,
    volume,
    ROR_n1,
    ROR_n3,
    ROR_n5,
    ROR_n10,
    ROR_n15,
    ROR_n20,
    ROR_n25,
    ROR_n30
    from `algomosaic-nyc.crypto_features.features_BTC_USD_day_20191222`
    """
)


Writing query to a dataframe.


Downloading: 100%|██████████| 4001/4001 [00:01<00:00, 2617.46rows/s]

Loaded query to dataframe in 0:00:03.122541.


In [3]:
from etl.talib_features import SMA
df = df.dropna()
df = SMA(df, n=3, value='ROR_n3')
df = SMA(df, n=5, value='ROR_n3')
df = SMA(df, n=3, value='ROR_n5')
df = SMA(df, n=5, value='ROR_n5')
df = SMA(df, n=3, value='ROR_n10')
df = SMA(df, n=5, value='ROR_n10')
df = SMA(df, n=10, value='ROR_n10')
df = SMA(df, n=3, value='ROR_n15')
df = SMA(df, n=5, value='ROR_n15')
df = SMA(df, n=10, value='ROR_n15')
print(len(df))
df.tail()


3417


,partition_date,ticker_datetime,ticker,exchange,interval,close,high,low,open,volume_base,...,SMA_ROR_n3_3,SMA_ROR_n3_5,SMA_ROR_n5_3,SMA_ROR_n5_5,SMA_ROR_n10_3,SMA_ROR_n10_5,SMA_ROR_n10_10,SMA_ROR_n15_3,SMA_ROR_n15_5,SMA_ROR_n15_10
3966,2019-12-22,2019-11-18 00:00:00+00:00,BTC-USD,CCCAGG,day,8190.16,8537.87,8075.63,8515.28,30048.32,...,0.050789,0.043942,0.109698,0.107514,0.122768,0.170020,0.151968,0.135691,0.113192,0.154250
3967,2019-12-22,2019-11-19 00:00:00+00:00,BTC-USD,CCCAGG,day,8133.57,8215.95,8016.54,8190.16,24402.60,...,0.070506,0.050789,0.155057,0.109698,0.095749,0.122768,0.151968,0.120839,0.120839,0.143407
3968,2019-12-22,2019-11-20 00:00:00+00:00,BTC-USD,CCCAGG,day,8093.60,8232.23,8047.73,8133.57,18105.11,...,0.097838,0.070506,0.125609,0.125609,0.066647,0.095749,0.151968,0.113192,0.120839,0.128265
3969,2019-12-22,2019-11-21 00:00:00+00:00,BTC-USD,CCCAGG,day,7632.59,8133.73,7437.33,8093.60,45760.44,...,0.097838,0.096554,0.125609,0.125609,0.045810,0.066647,0.146394,0.088682,0.113192,0.117016
3970,2019-12-22,2019-11-22 00:00:00+00:00,BTC-USD,CCCAGG,day,7292.20,7721.90,6815.03,7632.59,94065.31,...,0.096554,0.096554,0.062506,0.109698,0.028177,0.045810,0.109259,0.009744,0.088682,0.112564


In [9]:
# from trade_business_rules_BTC_USD import buy
# [eval('br.buy', locals={'data': v}) for k, v in data.items()]
# v = data[3412]
# eval('buy(v)')
# eval('buy', {'row': data[3412]})
# eval('from trade_business_rules_BTC_USD import buy')
# _module = importlib.import_module('trade_business_rules_BTC_USD')
# eval('_module.buy()')
# list(globals().keys())




In [7]:
d_out


{'trade': 'BUY', 'amount': 0.11482, 'commission': 0}

In [8]:

df_out.tail()



,partition_date,ticker_datetime,ticker,exchange,interval,close,high,low,open,volume_base,...,SMA_ROR_n10_3,SMA_ROR_n10_5,SMA_ROR_n10_10,SMA_ROR_n15_3,SMA_ROR_n15_5,SMA_ROR_n15_10,response,trade,amount,commission
3966,2019-12-22,2019-11-18 00:00:00+00:00,BTC-USD,CCCAGG,day,8190.16,8537.87,8075.63,8515.28,30048.32,...,0.122768,0.170020,0.151968,0.135691,0.113192,0.154250,"{'trade': 'BUY', 'amount': 0.1221, 'commission...",BUY,0.12210,0.0
3967,2019-12-22,2019-11-19 00:00:00+00:00,BTC-USD,CCCAGG,day,8133.57,8215.95,8016.54,8190.16,24402.60,...,0.095749,0.122768,0.151968,0.120839,0.120839,0.143407,"{'trade': 'BUY', 'amount': 0.12295, 'commissio...",BUY,0.12295,0.0
3968,2019-12-22,2019-11-20 00:00:00+00:00,BTC-USD,CCCAGG,day,8093.60,8232.23,8047.73,8133.57,18105.11,...,0.066647,0.095749,0.151968,0.113192,0.120839,0.128265,"{'trade': 'BUY', 'amount': 0.12355, 'commissio...",BUY,0.12355,0.0
3969,2019-12-22,2019-11-21 00:00:00+00:00,BTC-USD,CCCAGG,day,7632.59,8133.73,7437.33,8093.60,45760.44,...,0.045810,0.066647,0.146394,0.088682,0.113192,0.117016,"{'trade': 'BUY', 'amount': 0.13102, 'commissio...",BUY,0.13102,0.0
3970,2019-12-22,2019-11-22 00:00:00+00:00,BTC-USD,CCCAGG,day,7292.20,7721.90,6815.03,7632.59,94065.31,...,0.028177,0.045810,0.109259,0.009744,0.088682,0.112564,"{'trade': 'BUY', 'amount': 0.06857, 'commissio...",BUY,0.06857,0.0
